In [1]:
#import engie - responsible for creating connections
from sqlalchemy import create_engine, Column, Integer, String, or_, func, ForeignKey, select
# declarative base used to create models
from sqlalchemy.orm import declarative_base, sessionmaker, relationship, mapped_column, Mapped, DeclarativeBase
from typing import Optional
# common syntax for the connection url
# "<dialect>+<driver>://<usermname>:<password>@<host>:<port>/database"
# postfresql
# "postgresql+psycopg2://<username>:<password>@<hostname>:<port>/database"
# mysql
# "mysql://<username>:<password>@<hostname>:<port>/database"
#sqllite
# /// - relative path & //// - absolute path
db_url = "sqlite:///CivilPM.db"
engine = create_engine(db_url)
# Session maker
Session = sessionmaker(bind=engine)
# initiate session to create actions
session = Session()
# Base = DeclarativeBase()

In [2]:
class Base(DeclarativeBase):
    pass

class User(Base): 
    __tablename__ = "users"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    email: Mapped[str]
    password: Mapped[Optional[str]]
    boq_list_id: Mapped[list["BOQ"]] = relationship()    

class BOQ(Base):
    __tablename__ = "boqs"
    id: Mapped[int] = mapped_column(primary_key=True)
    client_id: Mapped["Client"] = relationship()   
    category: Mapped["Category"]  = relationship()  
    stages: Mapped[list[str]] = relationship()
    resources: Mapped[list[str]] = relationship()
    quantity: Mapped[Optional[str]]
    unit: Mapped[Optional[str]]
    rate: Mapped[Optional[float]]
    amount: Mapped[Optional[float]]
    user_id: Mapped[Optional[int]] = mapped_column(ForeignKey("users.id"))

class Client(Base):
    __tablename__ = "client_data"
    id: Mapped[int] = mapped_column(primary_key=True)
    client_name: Mapped[Optional[str]]
    total_sqft: Mapped[Optional[float]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))

class Category(Base):
    __tablename__ = "categories"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))
  
class Stage(Base):
    __tablename__ = "stages"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))

class Resource(Base):
    __tablename__ = "resources"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    category: Mapped[str]
    price: Mapped[float]
    unit: Mapped[str]    
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))


# this will create DB and tables
Base.metadata.create_all(engine)


In [3]:
# Create entries in table
user = User(name="Paneer", age=44)
user_2 = User(name="Radha",age=42)
user_3 = User(name="Neha", age=17)
task_1 = Task(title="Test task1", description="test description")
# add single object to session
session.add(user)
# add multiple objects to sesson
session.add_all([user_2,user_3, task_1])
user.tasks.extend([task_1])

#commit changes
session.commit()


user.tasks =[<__main__.Task object at 0x00000196A3BF7EC0>]


In [7]:
print(f"{user.tasks[0].title}")

Test task1


In [39]:
# Query table - ALL
all_users = session.query(User).all()

# filter results - return one or none
# users = session.query(User).filter_by(id=3).one_or_none()
# print(users.name)

# filter results
#users = session.query(User).filter_by(id=3).all() = no logical operators allowed
# users = session.query(User).filter(User.id >= 3).all() # using logical operators
# users = session.query(User).filter(User.id >= 3, User.name == 'Radha').all() # using multiple conditions

#using where method - like SQL
# users = session.query(User).where(or_(User.age >= 17, User.name == 'Radha')).all() # with or condition
users = session.query(User).where(((User.age >= 17) | (User.name == 'Radha')) &  (User.id == 6)).all() # combination of and and or
print(len(all_users)) # print lenth of all results

for user in users:
    print(f"ID: {user.id} - Name: {user.name} - Age: {user.age}")

6
ID: 6 - Name: Radha - Age: 42


In [19]:
# update records
user = session.query(User).filter_by(id=2).one_or_none()
print(user.name)
user.age=45
session.commit()

Paneer


In [21]:
#delete record
session.delete(user)
session.commit()

In [25]:
# ordering results

# users = session.query(User).order_by(User.age).all() # assending
users = session.query(User).order_by(User.age.desc(), User.name).all() # dessending
for user in users:
    print(f"ID: {user.id} - Name: {user.name} - Age: {user.age}")


ID: 1 - Name: Paneer - Age: 44
ID: 3 - Name: Radha - Age: 42
ID: 4 - Name: Neha - Age: 17


In [55]:
# group by 
# users = session.query(User).all()
users = session.query(User.age, func.count(User.id)).group_by(User.age).all()
print(users)
print(len(users))

[(17, 2), (42, 2), (44, 2)]
3


In [59]:
# chaining
users = session.query(User.age).filter(User.age > 24).filter(User.age < 50).all()
print(users)

users_tuple = (
    session.query(User.age, func.count(User.id))
    .filter(User.age > 24)
    .order_by(User.age)
    .filter(User.age < 50)
    .group_by(User.age)
    .all()
)

for age, count in users_tuple:
    print(f"Age: {age} - {count} users")

[(44,), (42,), (44,), (42,)]
Age: 42 - 2 users
Age: 44 - 2 users


In [ ]:
# one to many relationship

# unmapped method

class BaseModel(Base):
    __abstract__ = True
    __allow_unmapped__ = True

    id = Column(Integer, primary_key=True)

class Address(BaseModel):
    __tablename__ = "adressess"
    city = Column(String)
    state = Column(String)
    zip_code = Column(Integer)
    user_id = Column(ForeignKey("nusers.id"))

    def __repr__(self):
        return f"<Address(id={self.id}, city='{self.city}')>"

class Nuser(BaseModel):
    __tablename__ = "nusers"
    name = Column(String)
    age = Column(Integer)
    adresses = relationship(Address)

    def __repr__(self):
        return f"<Nuser(id={self.id}, username='{self.name}')>"

Base.metadata.create_all(engine)

In [3]:
import pandas as pd
df = pd.read_csv("C:\\work\\Streamlit_Code\\Streamlit\\test.csv", index_col="Item")
df

,Description,Grade,Length feet,Length inch,Width feet,Width inch,Height feet,Height inch,Rod size,Rod count,Ring size
Item,,,,,,,,,,,
Footing 1,NaN,M20,2.0,0.0,2.0,0.0,1.0,0.0,16 mm,4,NaN
Footing 2,NaN,M20,2.0,0.0,2.0,0.0,1.0,0.0,16 mm,4,NaN
Column 1,NaN,M20,1.0,6.0,1.0,6.0,10.0,0.0,16 mm & 20 mm,2 + 4,10 mm
Column 2,NaN,M20,1.0,0.0,1.0,0.0,10.0,0.0,12 mm & 16 mm,4,12 mm
Beam 1,NaN,M20,1.0,0.0,1.0,0.0,1.0,0.0,16 mm,4,NaN
Wall Footing 1,NaN,M20,1.0,0.0,1.0,0.0,1.0,0.0,16 mm,4,NaN


In [19]:
column_df = df.loc[df.index.str.startswith('Column')]
# print(column_df)
column_ring_length_list = []
column_rod_weight = {
"rod_8_mm": 0,
"rod_10_mm": 0,
"rod_12_mm": 0,
"rod_16_mm": 0, 
"rod_20_mm": 0, 
"rod_25_mm": 0
}

for index, seris in column_df.iterrows():
    rod_size_list = seris["Rod size"].replace(' mm','').split(' & ')    
    # print(rod_size_list)
    rod_quantity_list = seris["Rod count"].split(' + ')    
    rod_height_feet = seris["Height feet"]
    rod_height_inch_to_feet = seris["Height inch"]   / 12
    rod_height = rod_height_feet + rod_height_inch_to_feet
    ring_length = (seris["Length feet"] + (seris["Length inch"] / 12) + seris["Width feet"] + (seris["Width inch"] / 12) ) * 2
    ring_size = str(seris["Ring size"]).replace(' mm','')
    num_of_rings = rod_height / 0.5
    total_ring_length = ring_length * num_of_rings  
    s1 = rod_size_list[0]
    q1 = rod_quantity_list[0]                                            
    weight = ((int(s1)*int(s1)) /162.28) * (( rod_height / 3.281 ) * int(q1))
    column_rod_weight[f"rod_{s1}_mm"] += weight
    if len(rod_size_list) > 1:
        s2 = rod_size_list[1]
        if len(rod_quantity_list) > 1: 
            q2 = rod_quantity_list[1] 
        else:
            q2 = q1                                           
        weight = ((int(s2)*int(s2)) /162.28) * (( rod_height / 3.281 ) * int(q2))
        column_rod_weight[f"rod_{s2}_mm"] += weight
    if ring_size != "nan":
        weight = ((int(ring_size)*int(ring_size)) /162.28) * (( total_ring_length / 3.281 ) * 1)
        column_rod_weight[f"rod_{ring_size}_mm"] += weight

     # print(f"ring_size - {ring_size}") 
    # print(f"num_of_rings - {num_of_rings}")   
    # print(f"total_ring_length - {total_ring_length}")    

print(column_rod_weight)

{'rod_8_mm': 0, 'rod_10_mm': 22.537721948668537, 'rod_12_mm': 32.45431960608269, 'rod_16_mm': 28.848284094295725, 'rod_20_mm': 30.050295931558047, 'rod_25_mm': 0}


In [20]:
other_df = df.loc[~df.index.str.startswith('Column')]
# print(other_df)
other_ring_length_list = []
others_rod_weight = {
"rod_8_mm": 0,
"rod_10_mm": 0,
"rod_12_mm": 0,
"rod_16_mm": 0, 
"rod_20_mm": 0, 
"rod_25_mm": 0
}

for index, seris in other_df.iterrows():
    rod_size_list = seris["Rod size"].replace(' mm','').split(' & ')    
    # print(rod_size_list)
    rod_quantity_list = seris["Rod count"].split(' + ')    
    rod_height_feet = seris["Length feet"]
    rod_height_inch_to_feet = seris["Length inch"]   / 12
    rod_height = rod_height_feet + rod_height_inch_to_feet
    # print(rod_height)
    ring_length = (seris["Length feet"] + (seris["Length inch"] / 12) + seris["Width feet"] + (seris["Width inch"] / 12) ) * 2
    # print(ring_length)
    ring_size = str(seris["Ring size"]).replace(' mm','')
    num_of_rings = rod_height / 0.5
    total_ring_length = ring_length * num_of_rings  
    s1 = rod_size_list[0]
    q1 = rod_quantity_list[0]                                            
    weight = ((int(s1)*int(s1)) /162.28) * (( rod_height / 3.281 ) * int(q1))
    # print(f"size:{s1}:length:{rod_height}:quantity:{q1}: weight: {weight}")
    others_rod_weight[f"rod_{s1}_mm"] += weight
    if len(rod_size_list) > 1:
        s2 = rod_size_list[1]
        if len(rod_quantity_list) > 1: 
            q2 = rod_quantity_list[1] 
        else:
            q2 = q1                                           
        weight = ((int(s2)*int(s2)) /162.28) * (( rod_height / 3.281 ) * int(q2))
        others_rod_weight[f"rod_{s2}_mm"] += weight
    if ring_size != "nan":
        weight = ((int(ring_size)*int(ring_size)) /162.28) * (( total_ring_length / 3.281 ) * 1)
        others_rod_weight[f"rod_{ring_size}_mm"] += weight

     # print(f"ring_size - {ring_size}") 
    # print(f"num_of_rings - {num_of_rings}")   
    # print(f"total_ring_length - {total_ring_length}")    


print(others_rod_weight)

{'rod_8_mm': 0, 'rod_10_mm': 0, 'rod_12_mm': 0, 'rod_16_mm': 11.539313637718287, 'rod_20_mm': 0, 'rod_25_mm': 0}


In [34]:
coloum_rod_keys = column_rod_weight.keys()
coloum_rod_values = column_rod_weight.values()
other_rod_keys = others_rod_weight.keys()
other_rod_values = others_rod_weight.values()

coulum_steel_data = {
"size": coloum_rod_keys,
"weight": coloum_rod_values
}
other_steel_data = {
"size": other_rod_keys,
"weight": other_rod_values
}
column_steel_df = pd.DataFrame(data=coulum_steel_data)
other_steel_df = pd.DataFrame(data=other_steel_data)
final_steal_weight_df = column_steel_df + other_steel_df
print(column_steel_df)
print(other_steel_df)
print(final_steal_weight_df)



        size     weight
0   rod_8_mm   0.000000
1  rod_10_mm  22.537722
2  rod_12_mm  32.454320
3  rod_16_mm  28.848284
4  rod_20_mm  30.050296
5  rod_25_mm   0.000000
        size     weight
0   rod_8_mm   0.000000
1  rod_10_mm   0.000000
2  rod_12_mm   0.000000
3  rod_16_mm  11.539314
4  rod_20_mm   0.000000
5  rod_25_mm   0.000000
                 size     weight
0    rod_8_mmrod_8_mm   0.000000
1  rod_10_mmrod_10_mm  22.537722
2  rod_12_mmrod_12_mm  32.454320
3  rod_16_mmrod_16_mm  40.387598
4  rod_20_mmrod_20_mm  30.050296
5  rod_25_mmrod_25_mm   0.000000


In [49]:
import pandas as pd
df = pd.read_csv("C:\\work\\Streamlit_Code\\Streamlit\\test.csv", index_col="Item")
df

,Wall Length feet,Wall Length inch,Wall Height feet,Wall Height inch,Wall thickness(inch),Brick size,Cement ratio,Plastering thickness
Item,,,,,,,,
Wall 1,10.0,6.0,11.0,6.0,9.0,9 x 4 x 3,1:5,12 mm
Wall 2,1.0,0.0,1.0,0.0,6.5,9 x 4 x 3,1:4,12 mm


In [48]:
# iterate all rows
for index, row in df.iterrows():
    wall_length_feet = row["Wall Length feet"]
    wall_length_inch = row["Wall Length inch"]
    wall_length_inch_to_feet = wall_length_inch / 12
    total_wall_length_feet = wall_length_feet + wall_length_inch_to_feet
    total_wall_length_meter = total_wall_length_feet / 3.281

    wall_height_feet = row["Wall Height feet"]
    wall_height_inch = row["Wall Height inch"]
    wall_height_inch_to_feet = wall_height_inch / 12
    total_wall_height_feet = wall_height_feet + wall_height_inch_to_feet
    total_wall_height_meter = total_wall_height_feet / 3.281

    brick_size = row["Brick size"].split(' x ')
    brick_length = int(brick_size[0]) / 39.37
    brick_width = int(brick_size[1]) / 39.37
    brick_height = int(brick_size[2]) / 39.37
    wall_thickness = row["Wall thickness(inch)"] / 39.37

    # Volume of brick masonary = wall_length(m) x wall_Height(m) x wall_thickness(m)
    brick_masonary_volume = total_wall_length_meter * total_wall_height_meter * wall_thickness
    #brick volume with mortar in meter
    brick_volume_with_mortar = (brick_length + 0.01) * (brick_width + 0.01) * (brick_height + 0.01)
    # number of bricks required = Volume of brick masonary / brick_volume_with_mortar
    number_of_bricks = brick_masonary_volume / brick_volume_with_mortar



    actual_volume_of_bricks_mortar = number_of_bricks * (brick_length * brick_width * brick_height )
    quantity_of_mortar = brick_masonary_volume - actual_volume_of_bricks_mortar
    quantity_of_mortar = quantity_of_mortar + (quantity_of_mortar * (15/100)) 
    quantity_of_mortar = quantity_of_mortar + (quantity_of_mortar * (25/100)) 

    cement_ratio = row["Cement ratio"].split(':')[0]
    sand_ratio = row["Cement ratio"].split(':')[1]    
    sum_of_ratio = int(cement_ratio) + int(sand_ratio)

    amount_of_cement = (int(cement_ratio)/int(sum_of_ratio)) * quantity_of_mortar
    num_of_cement_bags = amount_of_cement / 0.035


    amount_of_sand = (int(sand_ratio)/int(sum_of_ratio)) * quantity_of_mortar
    sand_in_kg = amount_of_sand * 1500
    print(number_of_bricks)
    print(num_of_cement_bags)
    print(sand_in_kg)

0.5626090689409009
703.2613361761262
0.005047562385253079
6.057074862303694
